In [2]:
#Imports & some predefines
import numpy as np
# import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import os
import glob
import pandas as pd
import pickle
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Use cuda if possible; otherwise eat some threads on the CPU

In [3]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder('Figures/SNR', transform=transform)
dataloaded = DataLoader(dataset, batch_size=32, shuffle=True)

In [6]:
#Read one of the data files #TODO: Read all csvs

features = ['center_freq', 'dist', 'h_dist', 'v_dist', 'avgPower', 'avgSnr',
            'freq_offset', 'avg_pl', 'aod_theta', 'aoa_theta', 'aoa_phi',
            'pitch', 'yaw', 'roll', 'vel_x', 'vel_y', 'vel_z', 'speed', 'avg_pl_rolling', 'avg_pl_ewma']

df = pd.read_csv("dataset/2023-12-15_15_41-results.csv", usecols = features)


#Need to ensure only valid data goes beyond here #TODO

data = df[features].values#[:100] Limit values be needed - ask

# Function to create sequences
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data)-seq_length):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, y = create_sequences(data, seq_length)

# Convert data to PyTorch tensors - no filter currently, reproduce everything
trainX = torch.tensor(X, dtype=torch.float32)
trainY = torch.tensor(y, dtype=torch.float32)


In [15]:
# #Copy of Aayam's LSMT class - might need to be altered?
# class LSTMModel(nn.Module):
#     def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
#         super(LSTMModel, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.layer_dim = layer_dim
#         self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, output_dim)

#     def forward(self, x, h0=None, c0=None):
#         # If hidden and cell states are not provided, initialize them as zeros
#         if h0 is None or c0 is None:
#             h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
#             c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)

#         # Forward pass through LSTM
#         out, (hn, cn) = self.lstm(x, (h0, c0))
#         out = self.fc(out[:, -1, :])  # Selecting the last output
#         return out, hn, cn


In [16]:
# # Initialize model, loss, and optimizer
# model = LSTMModel(input_dim=len(features), hidden_dim=50, layer_dim=1, output_dim=len(features))
# criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [17]:
# #Training loop - LSTM
# num_epochs = 4000
# h0, c0 = None, None  # Initialize hidden and cell states

# for epoch in range(num_epochs):
#     model.train()
#     optimizer.zero_grad()

#     # Forward pass
#     outputs, h0, c0 = model(trainX, h0, c0)

#     # Compute loss
#     loss = criterion(outputs, trainY)
#     loss.backward()
#     optimizer.step()

#     # Detach hidden and cell states to prevent backpropagation through the entire sequence
#     h0 = h0.detach()
#     c0 = c0.detach()

#     if (epoch+1) % 10 == 0:
#         print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [18]:
#TODO - validate outputs now that it should be for multiple values not just SNR

In [24]:
#Build the Generator
class Generator(nn.Module):

    def __init__(self, inputSize=6, outputSize =(256,256)): #Default input size is 6 (likely needs to be changed); output size is 800x800
        super(Generator, self).__init__() #We should be fine using Pytorch base model code

        self.outputSize = outputSize #Need to remember the output size for the forward function

        self.model = nn.Sequential(nn.Linear(inputSize, 128), #mapping input vector to 128 neurons; 
                                   nn.LeakyReLU(0.2, inplace=True), #Using inplace=True to save memory + it seems to be standard practice
                                   #start hidden layers
                                   nn.Linear(128,256), #Hidden Layer 1; Upscaling by factor of 2
                                   nn.BatchNorm1d(256), #Normalization
                                   nn.LeakyReLU(0.2, inplace=True), 
                                   nn.Linear(256, 512), #Hidden Layer 2; Upsacling by factor of 2; Last hidden layer for now
                                   nn.BatchNorm1d(512), #Normalization
                                   nn.LeakyReLU(0.2, inplace=True),
                                   nn.Linear(512, outputSize[0] * outputSize[1] * 3), #Output layer; *3 for RGB 
                                   nn.Tanh() #Normalization for output [-1, 1]
                                   )
    
    #Passes data into first layer & executes sequentially based upon params from self.model()
    def forward(self, noise):
        genImage =  self.model(noise) #Grab output tensor
        return genImage.view(-1, 3, self.outputSize[0], self.outputSize[1]) #Should produce a RGB image using the view function. Args: Batchsize(-1 means figure it out for me), numChannels, height, width

In [ ]:
#Build the Discriminator
class Discriminator(nn.Module):

    def __init__(self, inputSize=(256,256)): #Simple classifier returns [0,1]; 1 real
        super(Discriminator, self).__init__() #Use the base Pytorch discriminator

        self.inputSize = inputSize #Need to remember the input size for the forward function

        #in channels 3 = RGB data; stride 2 = downscale (1 for no downscale); padding 1 = keep the same size
        self.model = nn.Sequential(nn.Conv2d(in_channels =3, out_channels=64, kernel_size=4, stride=2, padding=1), #Convolutional layer 1
                                   nn.LeakyReLU(0.2, inplace=True), #Activation function

                                   nn.Conv2d(64, 128, 4, 2, 1), #Convolutional layer 2 [batchSize, 64, 128, 128]
                                   nn.BatchNorm2d(128), #Normalization
                                   nn.LeakyReLU(0.2, inplace=True), #Activation function

                                   nn.Conv2d(128, 256, 4, 2, 1), #Convolutional layer 3 [batchSize, 128, 64, 64]
                                   nn.BatchNorm2d(256), #Normalization
                                   nn.LeakyReLU(0.2, inplace=True), #Activation function
                                   
                                   nn.Conv2d(256, 512, 4, 2, 1), #Convolutional layer 4 [batchSize, 256, 32, 32]
                                   nn.BatchNorm2d(512), #Normalization
                                   nn.LeakyReLU(0.2, inplace=True), #Activation function

                                   nn.Conv2d(512, 1, 4, 1, 0), #Output layer [batchSize, 512, 16, 16]

                                   #Hypothetically we could flatten the output and use a linear layer to get the sigmoid
                                #    nn.Flatten(), #Flatten the output
                                #    nn.Linear(512 * 49 * 49, 1), #Linear layer to get the sigmoid

                                   nn.Sigmoid() #Turn the output into [0,1] [batchSize, 1, 13, 13]
        )
    
    #Passes data into first layer & executes sequentially based upon params from self.model() Implicitly called when given input.
    def forward(self, input):
        # Ensure the input has a channel dimension (batch_size, 1, 800, 800)
        if len(input.shape) == 3:  # JIC somehow we got greyscale images
            input = input.unsqueeze(1)  # Add channel dimension
        
        output = self.model(input) #[9,1,13,13]
        print(f"0: {output[0]}")
        print(f"1: {output[1]}")
        print(f"2: {output[2]}")
        print(f"3: {output[3]}")
        output = output.view(-1, 1) #Flatten the output
        return output

In [33]:
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder('Figures/SNR', transform=transform)
dataloaded = DataLoader(dataset, batch_size=32, shuffle=True)

#Hyper params
GANepochs = 5
GANlr = 0.0002 #Should probably be low

#Labels - DO NOT USE 1 or 0, it will be to ridged, smooth it by ~.025 - .2
realLabel = 0.9
fakeLabel = 0.1

#Loss function
lossFunc = nn.BCELoss()

#Creating the Discriminator
discrim = Discriminator().to(DEVICE)
dOptimizer = torch.optim.Adam(discrim.parameters(), lr=0.0002, betas=(0.5, 0.999)) #Lower B1 since models will fight eachother

#Creating the Generator
generator = Generator().to(DEVICE)
gOptimizer = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999)) #Lower B1 since models will fight eachother


#load data somehow
#It would be visualizations of the data no?

#static noise
#fixedNoise = torch.randn(10, len(features)).to(DEVICE)

for epoch in range(GANepochs):
    
    for i, (realSpectrogram, _) in enumerate(dataloaded):
        #realSpectrogram.shape = torch.Size([9, 3, 256, 256])
        
        #Discriminator Training
        realSpectrogram = realSpectrogram.to(DEVICE) #Shove it on the GPU
        
        real = torch.full((realSpectrogram.size(0), 1), realLabel, device=DEVICE)  # Ensure it has shape [batch_size, 1]
        fake = torch.full((realSpectrogram.size(0), 1), fakeLabel, device=DEVICE)  # Ensure it has shape [batch_size, 1]


        #Discrim on the real images
        dOptimizer.zero_grad() #Zero gradients before training begins
        output = discrim(realSpectrogram)
        dLossOnReal = lossFunc(output, real)

        #Discrim on fake data
        noise = torch.randn(realSpectrogram.size(0), len(features)).to(DEVICE)
        fakeSpectrogram = generator(noise)
        output = discrim(fakeSpectrogram.detach()) #Detach - not training the generator yet
        dLossOnFake = lossFunc(output, fake)

        #Combined Loss
        dLossTotal = dLossOnReal + dLossOnFake
        dLossTotal.backward()
        dOptimizer.step()
        
        #Train the generator
        gOptimizer.zero_grad() #Zero gradients before training begins
        output = discrim(fakeSpectrogram) #not calling deteach - training the generator
        output = output.view(-1) #Flatten the output to match label
        gLoss = lossFunc(output, real)
        gLoss.backward()
        gOptimizer.step()

        #output so we know something happened
        if (epoch+1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{GANepochs}], D Loss: {dLossTotal.item():.4f}, G Loss: {gLoss.item():.4f}")




0: tensor([[[0.4728, 0.5326, 0.4530, 0.5169, 0.5079, 0.5119, 0.4801, 0.4431,
          0.4309, 0.5242, 0.4076, 0.3619, 0.3728],
         [0.3898, 0.4609, 0.4798, 0.5161, 0.5370, 0.5151, 0.4677, 0.5196,
          0.4251, 0.6026, 0.3325, 0.3651, 0.4706],
         [0.5397, 0.5817, 0.5389, 0.5643, 0.5393, 0.5273, 0.5381, 0.5629,
          0.4761, 0.5206, 0.3132, 0.4870, 0.4953],
         [0.4624, 0.6827, 0.5225, 0.5316, 0.5315, 0.5315, 0.5328, 0.5339,
          0.5147, 0.4974, 0.3262, 0.3896, 0.4413],
         [0.5226, 0.5432, 0.5268, 0.5350, 0.5346, 0.5354, 0.5337, 0.5356,
          0.5185, 0.5494, 0.4618, 0.4518, 0.4339],
         [0.5442, 0.5933, 0.5828, 0.5473, 0.5473, 0.5456, 0.5391, 0.5365,
          0.4648, 0.5537, 0.3894, 0.3726, 0.3424],
         [0.5261, 0.5692, 0.6212, 0.5933, 0.5933, 0.5926, 0.5931, 0.5883,
          0.5243, 0.6258, 0.4491, 0.3654, 0.4214],
         [0.5464, 0.5864, 0.6380, 0.5863, 0.5863, 0.5864, 0.5910, 0.5877,
          0.5069, 0.5996, 0.4125, 0.3704, 0.4859

ValueError: Using a target size (torch.Size([9, 1])) that is different to the input size (torch.Size([1521, 1])) is deprecated. Please ensure they have the same size.